In [3]:
import gurobipy as gp
from gurobipy import GRB

def formulate_optimization_problem(N, P_min, P_max, F_min, F_max):
    # Create a new model
    m = gp.Model("EnergyEfficiencyOptimization")

    # Decision Variables
    p = m.addVars(N, vtype=GRB.CONTINUOUS, lb=P_min, ub=P_max, name="power")
    f = m.addVars(N, vtype=GRB.CONTINUOUS, lb=F_min, ub=F_max, name="frequency")
    z = m.addVar(vtype=GRB.CONTINUOUS, name="auxiliary")  # Auxiliary variable

    # Objective Function
    D = sum(p[i] * f[i] for i in range(N))  # Placeholder representation for processed data
    E_total = sum(p[i] + f[i] for i in range(N))  # Placeholder representation for total energy
    
    m.setObjective(z, GRB.MAXIMIZE)  # Adjusted objective

    # Constraints
    # Power constraints
    for i in range(N):
        m.addConstr(p[i] >= P_min, name=f"MinPower_Task_{i}")
        m.addConstr(p[i] <= P_max, name=f"MaxPower_Task_{i}")

    # Frequency constraints
    for i in range(N):
        m.addConstr(f[i] >= F_min, name=f"MinFreq_Task_{i}")
        m.addConstr(f[i] <= F_max, name=f"MaxFreq_Task_{i}")

    # Reformulated constraint for the division
    m.addConstr(z * E_total <= D, name="Reformulated_Objective")

    # Set NonConvex parameter to handle non-convex quadratic programs
    m.setParam(GRB.Param.NonConvex, 2)
    
    # Optimize the model
    m.optimize()
    
    # Return results
    if m.status == GRB.OPTIMAL:
        results = [(p[i].x, f[i].x) for i in range(N)]
        return results
    else:
        return "No solution found!"

# Sample parameters
N = 5  # For example, 5 tasks
P_min = 1  # Minimum power
P_max = 10  # Maximum power
F_min = 1  # Minimum frequency
F_max = 5  # Maximum frequency

# Get results
results = formulate_optimization_problem(N, P_min, P_max, F_min, F_max)
results


Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 11 columns and 20 nonzeros
Model fingerprint: 0x9d7ded69
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+01]
Presolve removed 20 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve removed 20 rows and 0 columns
Presolve time: 0.01s
Presolved: 31 rows, 26 columns, 95 nonzeros
Presolved model has 15 bilinear constraint(s)
Variable types: 26 continuous, 0 integer (0 binary)

Root relaxation: objective 1.262626e+01, 28 iterations, 0.00 seconds (0.00 work units)


[(10.0, 5.0), (10.0, 5.0), (10.0, 5.0), (10.0, 5.0), (10.0, 5.000000000000001)]